In [39]:
import pandas as pd
import sys


df = pd.read_csv("./dataset/Delaware.csv")
# df = df[df.REGION == "New York"]

df_use = df[["REGION", "YEAR", "WEEK", "ili_ratio"]]
df_use.index = range(len(df_use))

df_use = (df_use.loc[:, ["ili_ratio"]])  # 只用ili_ratio


In [41]:
df = pd.read_csv("./dataset/Delaware.csv")
# df = df[df.REGION == "Delaware"]

df_use = df[["REGION", "YEAR", "WEEK", "ili_ratio"]]
df_use.index = range(len(df_use))

df_use = (df_use.loc[:, ["ili_ratio"]])  # 只用ili_ratio
df_use

,ili_ratio
0,0.001001
1,0.003686
2,0.003370
3,0.004605
4,0.002223
...,...
624,0.011423
625,0.009824
626,0.009270
627,0.012565


In [42]:
df = pd.read_csv("./dataset/covid19/france_new_cases.csv")

df_use = df[["New_cases"]]
df_use = df_use[:660]
df_use.index = range(len(df_use))

df_use

,New_cases
0,73
1,133
2,184
3,107
4,410
...,...
655,48473
656,14684
657,72634
658,79567


In [43]:
df = pd.read_csv("./dataset/covid19/france_new_cases _temp.csv")

df_use = df[["New_cases", "tavg"]]
df_use = df_use[:660]
df_use.index = range(len(df_use))

df_use

,New_cases,tavg
0,73,9.1
1,133,7.4
2,184,7.8
3,107,8.9
4,410,8.5
...,...,...
655,48473,3.9
656,14684,2.5
657,72634,2.5
658,79567,6.7


In [113]:
df = pd.read_csv('./dataset/elec/oneminute.csv')

df_use = df[['p']]
df_use.index = range(len(df_use))
df_use = df_use[:500]

In [44]:
from torch.utils.data import Dataset as Dataset
from torch.utils.data import DataLoader as DataLoader
import numpy as np
import torch

class MyDataset(Dataset):
    def __init__(self, data):
        self.data = data

    def __getitem__(self, item):
        return self.data[item]

    def __len__(self):
        return len(self.data)

def nn_seq_us(batch_size, step):
    print('data processing...')
    dataset = df_use
    # split
    train = dataset[:int(len(dataset) * 0.6)]
    val = dataset[int(len(dataset) * 0.6):int(len(dataset) * 0.85)]
    test = dataset[int(len(dataset) * 0.85):len(dataset)]
    m, n = np.max(train[train.columns[0]]), np.min(train[train.columns[0]])

    def process(data, batch_size, shuffle):
        load = data[data.columns[:]]
        load = load.values
        data = data.values.tolist()
        load = (load - n) / (m - n)

        # load = data[data.columns[0]]
        # load = load.tolist()
        # data = data.values.tolist()
        # load = (load - n) / (m - n)

        seq = []
        for i in range(len(data) - step):
            train_seq = []
            train_label = []
            for j in range(i, i + step):
                x = [load[j]]
                train_seq.append(x)
            # for c in range(2, 8):
            #     train_seq.append(data[i + step][c])
            train_label.append(load[i + step])
            train_seq = torch.FloatTensor(train_seq)
            train_label = torch.FloatTensor(train_label).view(-1)
            seq.append((train_seq, train_label))

        # print(seq[-1])
        seq = MyDataset(seq)
        seq = DataLoader(dataset=seq, batch_size=batch_size, shuffle=shuffle, num_workers=0, drop_last=True)

        return seq

    data_train = process(train, batch_size, True)
    data_validate = process(val, batch_size, True)
    data_test = process(test, batch_size, False)

    return data_train, data_validate, data_test, m, n

In [45]:
import torch.nn as nn
class LSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size, batch_size, device):
        super().__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.output_size = output_size
        self.num_directions = 1  # 单向LSTM
        self.batch_size = batch_size
        self.lstm = nn.LSTM(self.input_size, self.hidden_size, self.num_layers, batch_first=True)
        self.linear = nn.Linear(self.hidden_size, self.output_size)

        self.device = device

    def forward(self, input_seq):
        batch_size, seq_len = input_seq.shape[0], input_seq.shape[1]
        h_0 = torch.randn(self.num_directions * self.num_layers, self.batch_size, self.hidden_size).to(self.device)
        c_0 = torch.randn(self.num_directions * self.num_layers, self.batch_size, self.hidden_size).to(self.device)
        # output(batch_size, seq_len, num_directions * hidden_size)
        output, _ = self.lstm(input_seq, (h_0, c_0))  # output(5, 30, 64)
        pred = self.linear(output)  # (5, 30, 1)
        pred = pred[:, -1, :]  # (5, 1)
        return pred


In [57]:
from torch.optim.lr_scheduler import StepLR as StepLR
from tqdm import tqdm
from copy import deepcopy

def get_val_loss(model, Val, loss_function, device):
    model.eval()
    val_loss = []
    for (seq, label) in Val:
        seq = seq.to(device)
        label = label.to(device)
        y_pred = model(seq)
        loss = loss_function(y_pred, label)
        val_loss.append(loss.item())

    return sum(val_loss)/len(val_loss)# MSE


def lstm_train(data_train, Val, path, input_size=1, hidden_size=10, num_layers=2, output_size = 1, batch_size = 5, optimizer = 'adam', max_epochs = 20, lr = 0.01, device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")):
    weight_decay = 0.0001
    step_size = 5
    gamma = 0.1


    model = LSTM(input_size, hidden_size, num_layers, output_size, batch_size=batch_size, device=device).to(device)

    loss_function = nn.MSELoss().to(device)
    if optimizer == 'adam':
        optimizer = torch.optim.Adam(model.parameters(), lr=lr,
                                     weight_decay=weight_decay)
    else:
        optimizer = torch.optim.SGD(model.parameters(), lr=lr,
                                    momentum=0.9, weight_decay=weight_decay)
    scheduler = StepLR(optimizer, step_size=step_size, gamma=gamma)
    # training
    min_epochs = 10
    best_model = None
    min_val_loss = 5

    min_train_loss = 10086
    loss = 10

    for epoch in range(max_epochs):
        train_loss = []
        for (seq, label) in data_train:
            seq = seq.to(device)
            seq.resize_(batch_size, step, input_size)    # (1, 18, 1, 2) -> (1, 18, 2)

            label = label.to(device)
            y_pred = model(seq)

            if epoch == 10:
                print(f'seq:{seq.size()}')
                print(f'seq:{label.size()}')
                print(f'y_pred:{y_pred.size()}')

            loss = loss_function(y_pred, label)
            train_loss.append(loss.item())
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        scheduler.step()

        # # validation
        # val_loss = get_val_loss(model, Val, loss_function, device)
        # if epoch > min_epochs and val_loss < min_val_loss:
        #     min_val_loss = val_loss
        #     best_model = deepcopy(model)
        #
        # if epoch % 10 == 0:
        #     print('epoch {:03d} train_loss {:.8f} val_loss {:.8f}'.format(epoch, np.mean(train_loss), val_loss))

        if epoch > min_epochs and loss < min_train_loss:
            min_train_loss = loss
            best_model = deepcopy(model)
            print(type(model))

        if epoch % 10 == 0:
            print('epoch {:03d} train_loss {:.8f}'.format(epoch, np.mean(train_loss)))

        model.train()

    state = {'models': best_model.state_dict()}
    torch.save(state, path)

In [58]:
from itertools import chain
import matplotlib.pyplot as plt
from scipy.interpolate import make_interp_spline

def get_mse(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.sum((y_true - y_pred) ** 2) / len(y_true)


def get_mape(y_true, y_pred):
    """
    Compute mean absolute percentage error (MAPE)
    """
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100
# def lstm_test(data_test, path, m, n, input_size = 1,  hidden_size = 10, num_layers = 2, output_size = 1, batch_size = 5, device = torch.device('cpu')):
def lstm_test(data_test, path, m, n, input_size, hidden_size, num_layers, output_size, batch_size, device):
    pred = []
    y = []
    print('loading models...')

    model = LSTM(input_size, hidden_size, num_layers, output_size, batch_size=batch_size, device=device).to(device)
    model.load_state_dict(torch.load(path)['models'])
    model.eval()
    print('predicting...')
    for (seq, target) in data_test:
        target = list(chain.from_iterable(target.data.tolist()))
        y.extend(target)
        seq = seq.to(device)
        seq.resize_(batch_size, step, input_size)
        with torch.no_grad():
            y_pred = model(seq)
            y_pred = list(chain.from_iterable(y_pred.data.tolist()))
            pred.extend(y_pred)

    y, pred = np.array(y), np.array(pred)
    y = (m - n) * y + n
    pred = (m - n) * pred + n
    print('mape:', get_mape(y, pred))
    print('mse:', get_mse(y, pred))

    # plot
    x = [i for i in range(1, y.shape[0] + 1)]
    x_smooth = np.linspace(np.min(x), np.max(x), 900)
    y_smooth = make_interp_spline(x, y)(x_smooth)
    plt.plot(x_smooth, y_smooth, c='green', marker='*', ms=1, alpha=0.75, label='true')

    y_smooth = make_interp_spline(x, pred)(x_smooth)
    plt.plot(x_smooth, y_smooth, c='red', marker='o', ms=1, alpha=0.75, label='pred')
    plt.grid(axis='y')
    plt.legend()
    # plt.savefig("./new.png")
    plt.show()


def future_lstm_test(data_test, path, m, n, input_size, hidden_size, num_layers, output_size, batch_size, device):
    pred = []
    y = []
    print('loading models...')

    model = LSTM(input_size, hidden_size, num_layers, output_size, batch_size=batch_size, device=device).to(device)
    model.load_state_dict(torch.load(path)['models'])
    model.eval()
    print('predicting...')


    y_pred = 0
    seq = []
    label = []
    ss = [(seq, target) for (seq, target) in data_test]
    for i in range(len(ss)):
        label.append(ss[i][1][0])

    y = label

    # label = list(chain.from_iterable(label.tolist()))
    ss = ss[0]
    # print(ss)
    for i in range(50):

        seq = ss[0].to(device)
        with torch.no_grad():
            y_pred = model(seq)
            # print(f'seq{seq};')
            for i in range(len(seq[0]) - 1):
                seq[0][i] = seq[0][i+1]
            seq[0][-1] = y_pred
            # print(f'pred: {y_pred}')
            y_pred = list(chain.from_iterable(y_pred.data.tolist()))
            pred.extend(y_pred)


    # print(f'y_pred is {y_pred}')

    y = []
    for i in range(len(label)):
        y.append(label[i].detach().numpy()[0])


    y, pred = np.array(y), np.array(pred)
    y = (m - n) * y + n
    pred = (m - n) * pred + n
    # print('mape:', get_mape(y, pred))
    # print('mse:', get_mse(y, pred))
    print(len(y))
    print(len(pred))

    # plot
    # x = [i for i in range(1, y.shape[0] + 1)]
    x = [i for i in range(1, 50+1)]
    x_smooth = np.linspace(np.min(x), np.max(x), 900)
    y_smooth = make_interp_spline(x, y[:50])(x_smooth)
    plt.plot(x_smooth, y_smooth, c='green', marker='*', ms=1, alpha=0.75, label='true')

    # x = [i for i in range(1, pred.shape[0] + 1)]
    x = [i for i in range(1, 50+1)]
    y_smooth = make_interp_spline(x, pred[:50])(x_smooth)
    plt.plot(x_smooth, y_smooth, c='red', marker='o', ms=1, alpha=0.75, label='pred')
    plt.grid(axis='y')
    plt.legend()
    # plt.savefig("./new.png")
    plt.show()

In [59]:
input_size=2
hidden_size=128
num_layers=2
output_size = 1
batch_size = 1
optimizer = 'adam'
max_epochs = 25
# lr = 0.05
lr = 0.01   # mape:14.~    train_loss: 0.0037~

step = 18

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

print(device)

model_path = './lstm_model/best_model_lstm.model'

data_train, data_validate, data_test, m, n = nn_seq_us(batch_size=batch_size, step=step)

lstm_train(data_train, data_validate, model_path, input_size, hidden_size, num_layers, output_size, batch_size, optimizer, max_epochs, lr, device)

cpu
data processing...


D:\soft\Miniconda\envs\pytorch_practice_py38\lib\site-packages\torch\nn\modules\loss.py:520: UserWarning: Using a target size (torch.Size([1, 2])) that is different to the input size (torch.Size([1, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


epoch 000 train_loss 0.02575524
seq:torch.Size([1, 18, 2])
seq:torch.Size([1, 2])
y_pred:torch.Size([1, 1])
seq:torch.Size([1, 18, 2])
seq:torch.Size([1, 2])
y_pred:torch.Size([1, 1])
seq:torch.Size([1, 18, 2])
seq:torch.Size([1, 2])
y_pred:torch.Size([1, 1])
seq:torch.Size([1, 18, 2])
seq:torch.Size([1, 2])
y_pred:torch.Size([1, 1])
seq:torch.Size([1, 18, 2])
seq:torch.Size([1, 2])
y_pred:torch.Size([1, 1])
seq:torch.Size([1, 18, 2])
seq:torch.Size([1, 2])
y_pred:torch.Size([1, 1])
seq:torch.Size([1, 18, 2])
seq:torch.Size([1, 2])
y_pred:torch.Size([1, 1])
seq:torch.Size([1, 18, 2])
seq:torch.Size([1, 2])
y_pred:torch.Size([1, 1])
seq:torch.Size([1, 18, 2])
seq:torch.Size([1, 2])
y_pred:torch.Size([1, 1])
seq:torch.Size([1, 18, 2])
seq:torch.Size([1, 2])
y_pred:torch.Size([1, 1])
seq:torch.Size([1, 18, 2])
seq:torch.Size([1, 2])
y_pred:torch.Size([1, 1])
seq:torch.Size([1, 18, 2])
seq:torch.Size([1, 2])
y_pred:torch.Size([1, 1])
seq:torch.Size([1, 18, 2])
seq:torch.Size([1, 2])
y_pred

In [32]:
lstm_test(data_test, model_path, m, n, input_size, hidden_size, num_layers, output_size, batch_size, device)

loading models...
predicting...


ValueError: operands could not be broadcast together with shapes (162,) (81,) 

In [79]:
data_train.__len__(), data_validate.__len__()

(378, 147)

In [28]:
future_lstm_test(data_test, model_path, m, n, input_size, hidden_size, num_layers, output_size, batch_size, device)

loading models...
predicting...


RuntimeError: input must have 3 dimensions, got 4

In [72]:
def list_of_groups(data, sub_len):
    groups = zip(*(iter(data),) * sub_len)
    end_list = [list(i) for i in groups]
    count = len(data) % sub_len
    end_list.append(data[-count:]) if count != 0 else end_list
    return end_list


In [77]:
test = [(x, label) for (x, label) in iter(data_test)]
test[0:1]

[(tensor([[[0.8111],
           [0.7944],
           [0.8977],
           [0.8184],
           [0.8935],
           [0.8894],
           [0.9812],
           [1.0522],
           [0.9958],
           [1.2495],
           [1.3382],
           [1.4509],
           [1.4301],
           [1.6722],
           [1.5783],
           [1.5511],
           [1.4363],
           [1.3111],
           [1.3100],
           [1.2161],
           [1.3288],
           [1.4019],
           [1.2129],
           [1.4154]]]),
  tensor([[1.4415]]))]

In [12]:
def ss_rolling_test(data_test, input_size, hidden_size, num_layers, output_size, batch_size, device, path, m, n, pred_step_size):
    pred = []
    y = []

    model = LSTM(input_size, hidden_size, num_layers, output_size, device=device, batch_size=batch_size).to(device)
    model.load_state_dict(torch.load(path)['models'])
    model.eval()

    print('predicting...')

    data_test = [x for x in iter(data_test)]

    data_test = list_of_groups(data_test, pred_step_size)

    for sub_item in data_test:
        sub_pred = []
        for seq_idx, (seq, label) in enumerate(sub_item, 0):
            label = list(chain.from_iterable(label.data.tolist()))
            y.extend(label)
            # print(y)
            if seq_idx != 0:
                seq = seq.cpu().numpy().tolist()[0]
                if len(sub_pred) >= len(seq):
                    for t in range(len(seq)):
                        seq[t][0] = sub_pred[len(sub_pred) - len(seq) + t]
                else:
                    for t in range(len(sub_pred)):
                        seq[len(seq) - len(sub_pred) + t][0] = sub_pred[t]
            else:
                seq = seq.cpu().numpy().tolist()[0]


            # print(new_seq)
            seq = [seq]
            seq = torch.FloatTensor(seq)
            seq = MyDataset(seq)
            seq = DataLoader(dataset=seq, batch_size=1, shuffle=False, num_workers=0)
            # print(new_seq)
            seq = [x for x in iter(seq)][0]
            # print(new_seq)
            with torch.no_grad():
                seq = seq.to(device)
                y_pred = model(seq)
                y_pred = list(chain.from_iterable(y_pred.data.tolist()))
                # print(y_pred)
                sub_pred.extend(y_pred)


ss_rolling_test(data_test, input_size, hidden_size, num_layers, output_size, batch_size, device, "./practice/covid19/lstm_model/best_model_lstm.model", m, n, 12)

FileNotFoundError: [Errno 2] No such file or directory: './practice/covid19/lstm_model/best_model_lstm.model'